In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

# Loading data

In [7]:
df_pro_contacts_adults = pd.read_csv("pro_contacts_adults.csv", index_col=0)
df_pro_contacts_children = pd.read_csv("pro_contacts_children.csv", index_col=0)
df_households = pd.read_csv("households.csv", index_col=0)

# Initializing the graph

We start by defining the classes whose instances will be the nodes of our graph

In [29]:
class Person():
    def __init__(self, person_id, age):
        self.person_id = person_id
        self.age = age

class Adult(Person):
    def __init__(self, person_id, job_cat, pro_contacts, company_id):
        super().__init__(person_id, "adult")
        self.job_cat = job_cat
        self.pro_contacts = pro_contacts
        self.company_id = company_id

class Child(Person):
    def __init__(self, person_id, school_contacts, school_id):
        super().__init__(person_id, "child")
        self.job_cat = job_cat
        self.school_contacts = school_contacts
        self.school_id = school_id


We create an empty graph into which we add nodes from the pro_contacts dataframes.

In [40]:
G = nx.Graph()
for i, row in df_pro_contacts_adults.iterrows(): 
    person_id = row.adult_id
    job_cat = row.job_cat
    pro_contacts = row.pro_contacts
    company_id = row.company_id
    
    adult_node = Adult(person_id, job_cat, pro_contacts, company_id)
    G.add_node(adult_node)
    
for i, row in df_pro_contacts_children.iterrows(): 
    person_id = row.child_id
    school_contacts = row.school_contacts
    school_id = row.school_id
    
    child_node = Child(person_id, school_contacts, school_id)
    G.add_node(child_node)